In [20]:
import pandas as pd

solar_data = pd.read_csv("./historicData/Postcode data for small-scale installations - SGU-Solar.csv", index_col="Small Unit Installation Postcode")
solar_data.index.name = "postcode"
columns = solar_data.columns

print("saving postcode totals ...")
postcode_total = solar_data[["Installations Quantity Total"]]
postcode_total.columns = ["installations"]
postcode_total.to_csv("./solarData/postcode_total.csv")

print("create postcode list ...")
data_postcodes = list(solar_data.index)

print("calculate valid postcodes ...")
poa = pd.read_csv("./valid_postcodes.csv")
valid_postcodes = list(poa["postcode"])

postcodes = []
for postcode in data_postcodes:
    if postcode in valid_postcodes:
        postcodes.append(postcode)
pd.Series(postcodes).to_csv("./filtered_postcodes.csv")

solar_time_series = pd.DataFrame(index=solar_data.index)

for year in range(2001, 2021):
    print(f"calculating {year} time-series data ...")
    if year > 2018:
        data_url = "./historicData/Postcode data for small-scale installations - SGU-Solar.csv"
    else:
        data_url = f"./historicData/Postcode data for small-scale installations {year} - SGU-Solar.csv"
    temp_data = pd.read_csv(data_url, index_col="Small Unit Installation Postcode")    
    filtered_columns = []
    for column in temp_data.columns:
        if f"{year} - Installations Quantity" in column:
            filtered_columns.append(column)
    temp_data = temp_data[filtered_columns]
    for column in temp_data.columns:
        solar_time_series[column[:8]] = temp_data[column]

print("saving monthly installations ...")
monthly_installs = pd.DataFrame(solar_time_series.sum())
monthly_installs.columns = ["installations"]
monthly_installs.to_csv("./solarData/monthly_installs.csv", index_label="month")

print("saving time-series data ...")
solar_time_series = solar_time_series.cumsum(axis=1)
solar_time_series.to_csv("./solarData/solar_timeseries.csv")

df1 = pd.read_csv("./solarData/solarPostcodeIncome2016house.csv", index_col="postcode")
df2 = pd.read_csv("./solarData/solarPostcodeIncome2016semi1.csv", index_col="postcode")
df3 = pd.read_csv("./solarData/solarPostcodeIncome2016semi2.csv", index_col="postcode")
solar_incomes = df1 + df2 + df3
solar_incomes.index = pd.Series(solar_incomes.index).apply(lambda x: x[:4])

df1 = pd.read_csv("./solarData/solarPostcodeTenure2016house.csv", index_col="postcode")
df2 = pd.read_csv("./solarData/solarPostcodeTenure2016semi1.csv", index_col="postcode")
df3 = pd.read_csv("./solarData/solarPostcodeTenure2016semi2.csv", index_col="postcode")
solar_tenure = df1 + df2 + df3
solar_tenure.index = pd.Series(solar_tenure.index).apply(lambda x: x[:4])

postcode_check = []
for postcode in solar_incomes.index:
    if int(postcode) in postcodes:
        postcode_check.append(postcode)
        
solar_incomes = solar_incomes.loc[postcode_check]
solar_incomes = solar_incomes[solar_incomes.columns[:-4]]
solar_incomes["nil"] = solar_incomes["Negative income"] + solar_incomes["Nil income"]
solar_incomes = solar_incomes[["nil"] + list(solar_incomes.columns[2:-1])]
solar_incomes.to_csv("./solarData/solar_incomes.csv")

solar_tenure = solar_tenure.loc[postcode_check]
solar_tenure["rented"] = solar_tenure["Rented"] + solar_tenure["Being occupied rent-free"] + solar_tenure["Being occupied under a life tenure scheme"]
solar_tenure = solar_tenure[list(solar_tenure.columns[:2]) + ["rented"]]
solar_tenure.columns = ["outright", "mortgage", "rent"]
solar_tenure.to_csv("./solarData/solar_tenure.csv")

saving postcode totals ...
create postcode list ...
calculate valid postcodes ...
calculating 2001 time-series data ...
calculating 2002 time-series data ...
calculating 2003 time-series data ...
calculating 2004 time-series data ...
calculating 2005 time-series data ...
calculating 2006 time-series data ...
calculating 2007 time-series data ...
calculating 2008 time-series data ...
calculating 2009 time-series data ...
calculating 2010 time-series data ...
calculating 2011 time-series data ...
calculating 2012 time-series data ...
calculating 2013 time-series data ...
calculating 2014 time-series data ...
calculating 2015 time-series data ...
calculating 2016 time-series data ...
calculating 2017 time-series data ...
calculating 2018 time-series data ...
calculating 2019 time-series data ...
calculating 2020 time-series data ...
saving monthly installations ...
saving time-series data ...
